## Comprehensive Guide on Feature Selection

- https://www.kaggle.com/code/prashant111/comprehensive-guide-on-feature-selection

In [1]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy import stats
import warnings; warnings.filterwarnings('ignore')
#plt.style.use('ggplot')
plt.style.use('seaborn-whitegrid')
%matplotlib inline

### 1. Introduction to Feature Selection

feature selection은 ML 모델을 학습하기 위해 전체 피처에서 피처 서브셋을 선택하는 과정이다.


#### Advantages of selecting features
- 정확도 개선
- 모델이 심플해져서 해석이 용의
- 학습시간 단축
- 오버피팅을 완화해서 일반화 성능 향상
- 소프트웨어 구현이 용이
- 배포중 모델 에러 위험 저감

#### Feature selection 기법
- Filter methods
  - Basic methods
  - Univariate methods
  - Information gain
  - Fischer score
  - Correlation Matrix with Heatmap
- Wrapper methods
  - Forward Selection
  - Backward Elimination
  - Exhaustive Feature Selection
  - Recursive Feature Elimination
  - Recursive Feature Elimination with Cross-Validation
- Embedded methods
  - LASSO
  - RIDGE
  - Tree Importance

### 2.Filter methods
필터 방법은 전처리 단계로 사용되며, ML 방법과 무관함.  
피처를 선태할 때 변수들의 상관도를 통계 기법으로 테스트해서 선택한다.   
이 방법의 특징은 다음과 같음
- 데이터, 즉 피처의 특성에 의존적이다.
- ML 모델을 사용하지 않는다.  
- 계산 비용이 적다.
- ML 모델 종류에 구애받지 않는다.(Model agnostic)
- 일반적으로 wrapper 방식보다 예측 성능이 낮다.  
- 관련없는 피처를 빠르게 제거하고 싶을 때 적합하다.

필터 방법의 종류는 다음과 같다.  
- 2.1. Basic methods
- 2.2. Univariate feature selection
- 2.3. Information gain
- 2.4. Fischer score
- 2.5. ANOVA F-Value for Feature Selection
- 2.6. Correlation Matrix with Heatmap

필터방식 적용절차를 표현하면 다음과 같다.  
set of all features -> selecting the best subset -> learning algorithm -> performance

#### 2.1 Basic methods   
- 상수, 준 상수 피처를 제거한다.

상수피처 제거하기 
- 여기서는 sklearn의 VarianceThreshold function을 사용함
- https://scikit-learn.org/stable/modules/feature_selection.html
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html

In [2]:
os.listdir('data')

['mushrooms', 'house-price', 'santander', 'house', 'vehicles.csv']

In [3]:
os.walk('data')

<generator object _walk at 0x1470d9d60>

In [4]:
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data/vehicles.csv
data/mushrooms/mushrooms.csv
data/house-price/test.csv
data/house-price/data_description.txt
data/house-price/train.csv
data/house-price/sample_submission.csv
data/santander/test.csv
data/santander/train.csv
data/santander/sample_submission.csv
data/house/train.csv


산탄데르 데이터셋 사용

In [5]:
X_train = pd.read_csv('data/santander/train.csv', nrows = 35000)
X_test = pd.read_csv('data/santander/test.csv', nrows=15000)

In [6]:
X_train.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [7]:
X_train = X_train.drop(labels=['TARGET'], axis = 1)

In [8]:
X_train.shape, X_test.shape

((35000, 370), (15000, 370))

중요 : feature selection을 진행할 때 학습 데이터만 사용한다. 과적합 방지

variance threshold from sklearn
- Variance threshold는 간단한 베이스라인이 될 수 있다.  
- 이 방법은 Variance 가 특정 기준을 만족하지 못하면 피처를 제거하는 방식이다.
- 디펄트로 zero-variance 피처를 제거한다.(즉 모든 샘플에서 같은 값을 가지는 피처)

In [9]:
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=0)
sel.fit(X_train)

VarianceThreshold(threshold=0)

In [10]:
# get support 를 사용하면 유지해야할 피처수를 리턴한다.
sel.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False, False, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [11]:
sum(sel.get_support())

319

In [12]:
len(X_train.columns[sel.get_support()])

319

In [13]:
# 상수 컬럼 확인
print(
    len([
        x for x in X_train.columns
        if x not in X_train.columns[sel.get_support()]
    ])
)

51


In [14]:
[x for x in X_train.columns if x not in X_train.columns[sel.get_support()]]

['ind_var2_0',
 'ind_var2',
 'ind_var18_0',
 'ind_var18',
 'ind_var27_0',
 'ind_var28_0',
 'ind_var28',
 'ind_var27',
 'ind_var34_0',
 'ind_var34',
 'ind_var41',
 'ind_var46_0',
 'ind_var46',
 'num_var18_0',
 'num_var18',
 'num_var27_0',
 'num_var28_0',
 'num_var28',
 'num_var27',
 'num_var34_0',
 'num_var34',
 'num_var41',
 'num_var46_0',
 'num_var46',
 'saldo_var18',
 'saldo_var28',
 'saldo_var27',
 'saldo_var34',
 'saldo_var41',
 'saldo_var46',
 'delta_imp_amort_var18_1y3',
 'delta_imp_amort_var34_1y3',
 'imp_amort_var18_hace3',
 'imp_amort_var18_ult1',
 'imp_amort_var34_hace3',
 'imp_amort_var34_ult1',
 'imp_reemb_var13_hace3',
 'imp_reemb_var17_hace3',
 'imp_reemb_var33_hace3',
 'imp_trasp_var17_out_hace3',
 'imp_trasp_var33_out_hace3',
 'num_var2_0_ult1',
 'num_var2_ult1',
 'num_reemb_var13_hace3',
 'num_reemb_var17_hace3',
 'num_reemb_var33_hace3',
 'num_trasp_var17_out_hace3',
 'num_trasp_var33_out_hace3',
 'saldo_var2_ult1',
 'saldo_medio_var13_medio_hace3',
 'saldo_medio_var2

transform을 사용하면 해당 피처를 제거한다.

In [15]:
X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

X_train.shape, X_test.shape

((35000, 319), (15000, 319))

준 상수 피처(quasi-constant feature) 제거

- 준상수 피처는 측정 값의 대부분이 동일한 값을 나타내는 피처이다.   
- 일반적으로 ML모델을 학습하는데 정보를 거의 제공하지 않는다. 하지만 예외도 있으므로 제거할 때 유의해야한다.  
- sklearn의 VarianceThreshold를 조정하여 사용가능하다.

In [16]:
X_train = pd.read_csv('data/santander/train.csv', nrows = 35000)
X_test = pd.read_csv('data/santander/test.csv', nrows=15000)

X_train = X_train.drop(labels=['TARGET'], axis = 1)

X_train.shape, X_test.shape

((35000, 370), (15000, 370))

variance 임계 값을 설정하여 요구조건을 충족하지 않는 것을 제거한다.

In [18]:
sel = VarianceThreshold(threshold=0.01)  # 데이터의 99% 이상이 일정하면 제거

sel.fit(X_train) 

VarianceThreshold(threshold=0.01)

In [19]:
sum(sel.get_support())

263

In [20]:
len(X_train.columns[sel.get_support()])

263

In [21]:
# finally we can print the quasi-constant features
print(
    len([
        x for x in X_train.columns
        if x not in X_train.columns[sel.get_support()]
    ]))

[x for x in X_train.columns if x not in X_train.columns[sel.get_support()]]

107


['ind_var1',
 'ind_var2_0',
 'ind_var2',
 'ind_var6_0',
 'ind_var6',
 'ind_var13_largo',
 'ind_var13_medio_0',
 'ind_var13_medio',
 'ind_var14',
 'ind_var17_0',
 'ind_var17',
 'ind_var18_0',
 'ind_var18',
 'ind_var19',
 'ind_var20_0',
 'ind_var20',
 'ind_var27_0',
 'ind_var28_0',
 'ind_var28',
 'ind_var27',
 'ind_var29_0',
 'ind_var29',
 'ind_var30_0',
 'ind_var31_0',
 'ind_var31',
 'ind_var32_cte',
 'ind_var32_0',
 'ind_var32',
 'ind_var33_0',
 'ind_var33',
 'ind_var34_0',
 'ind_var34',
 'ind_var40',
 'ind_var41',
 'ind_var39',
 'ind_var44_0',
 'ind_var44',
 'ind_var46_0',
 'ind_var46',
 'num_var6_0',
 'num_var6',
 'num_var13_medio_0',
 'num_var13_medio',
 'num_var18_0',
 'num_var18',
 'num_op_var40_hace3',
 'num_var27_0',
 'num_var28_0',
 'num_var28',
 'num_var27',
 'num_var29_0',
 'num_var29',
 'num_var33',
 'num_var34_0',
 'num_var34',
 'num_var41',
 'num_var46_0',
 'num_var46',
 'saldo_var18',
 'saldo_var28',
 'saldo_var27',
 'saldo_var34',
 'saldo_var41',
 'saldo_var46',
 'delta_

- 107개의 컬럼이 상수에 가깝다.  
- 즉 샘플의 99% 이상이 같은 값인 컬럼이 107개이다.

In [22]:
# 컬럼 예시
X_train['ind_var31'].value_counts() / len(X_train)

0    0.996286
1    0.003714
Name: ind_var31, dtype: float64

In [23]:
X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

X_train.shape, X_test.shape

((35000, 263), (15000, 263))

- 상수, 준상수 피처를 제거기법을 통해서 100개 이상의 피처를 줄일수 있다
- 370 -> 263

#### 2.2 Univariate selection methods

- ANOVA와 같은 단변량 통계 테스트를 기반으로 최상의 피처를 선택하는 방법이다.  
- 이 방법도 전처리 방법의 일환이다.  
- F-test에 기반한 방법은 두 변수간의 선형 의존성의 정도를 추정한다. 이 방법은 더 피처와 타겟 간의 선형 관계가 있다고 가정한다. 그리고 피처들이 가우시안을 따른다고 가정한다.

대표적인 방법은 다음과 같다.  
- SelectKBest
- SelectPercentile
- SelectFpr, SelectFdr, or family wise error SelectFwe
- GenericUnivariateSelection

- https://scikit-learn.org/stable/modules/feature_selection.html

SelectKBest  
k highest scores에 기반하여 피처를 선택하는 방법이다.  
예를 들어 chi-squre(카이제곱) 테스트를 수행해서 아이리스 데이터의 가장 우사한 피처 두개를 찾아낼 수 있다.
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest

In [25]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest, chi2

X, y = load_iris(return_X_y=True)
X.shape

(150, 4)

In [26]:

X_new = SelectKBest(score_func=chi2, k=2).fit_transform(X, y)
X_new.shape

(150, 2)

SelectPercentile  
highest scores의 퍼센타일에 따라 스코어 선택???
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html#sklearn.feature_selection.SelectPercentile

In [28]:
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectPercentile, chi2

X, y = load_digits(return_X_y=True)
X.shape

(1797, 64)

In [30]:
# top 10 percentile 피처를 선택

X_new = SelectPercentile(score_func=chi2, percentile=10).fit_transform(X, y)
X_new.shape

(1797, 7)

- SelectKBest, SelectPercentile 객체는 univariate score나 P-value를 리턴하는 score function을 입력으로 받는다.
- 회귀 문제에서는 :  f_regression, mutual_info_regression
- 분류 문제에서는 : chi2, f_classif, mutual_info_classif

위에서,
- F-test에 기반한 방법은 두 랜덤변수간의 선형 의존성의 정도를 추정한다.   
- 반면 mutual information 방법은 모든 종류의 통계적 의존성을 포착한다. nonparametic이어서 정확한 추정을 위해 많은 샘플이 필요하다.

Feature selection with sparse data  
- sparse data를 다룬다면, chi2, mutual_info_regression, mutual_info_classif 를 추천한다..??  
- https://scikit-learn.org/stable/modules/feature_selection.html

#### Information Gain

- Information gain 또는 mutual information은 피처의 유무가 얼마나 정확한 예측에 기여하는지 측정한다.  
- mutual information은 X, Y가 공유하는 정보를 측정한다. 
  - 즉, 두 변수 중 하나를 알면 다른 변수에 대한 불확실성이 얼마나 줄어드는지를 측정합니다.  
  - 예를 들어 X와 Y가 독립적인 경우 X를 안다고 해서 Y에 대한 정보를 얻을 수 없으며, 그 반대의 경우도 마찬가지이므로 상호 정보는 0입니다.  
  - 반대로 X가 Y의 결정 함수이고 Y가 X의 결정 함수인 경우 X가 전달하는 모든 정보는 Y와 공유되므로 X를 알면 Y의 값이 결정되고 그 반대의 경우도 마찬가지입니다. 결과적으로 이 경우 상호 정보는 Y(또는 X)에만 포함된 불확실성, 즉 Y(또는 X)의 엔트로피와 동일합니다. 또한 이 상호 정보는 X의 엔트로피와 Y의 엔트로피와 동일합니다. (매우 특별한 경우는 X와 Y가 동일한 무작위 변수인 경우입니다.)

mutual_info_classif
- 분류용 타겟 변수에 대한 상호 정보를 측정
- 두 랜덤 변수간의 mutual info는 음수가 아닌 값이며, 두 변수간 의존성을 측정함. 갚이 클수록 의존성이 높음
- 이 함수는 KNN distance의 엔트로피 추정에 기반한 nonparametic 방법을 사용한다.
- 단변량 피처 선택에 사용할 수 있다. 

mutual_info_regression
- 회귀용 타겟 변수에 대한 상호 정보를 측정
- 두 랜덤 변수간의 mutual info는 음수가 아닌 값이며, 두 변수간 의존성을 측정함. 갚이 클수록 의존성이 높음
- 이 함수는 KNN distance의 엔트로피 추정에 기반한 nonparametic 방법을 사용한다.
- 단변량 피처 선택에 사용할 수 있다. 

#### 2.4 Fisher Score (chi-square implementation) 

- 사이킷런에서 chi-squre를 구현한 것임. 음수가 아닌 각 특징과 클래스 간의 카이제곱 통계를 계산함.
- 이 score는 분류 문제에서 카테고리 변수를 평가하는데 사용해야한다. 
- 이 score는 피처의 여러 카테고리 중에서 타겟 Y의 여러 클래스에서 관찰된 분포를 피처 카테고리와 관계없이 예측된 타겟 분포와 비교한다.

In [31]:
# load libraries
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest, chi2


In [34]:
iris = load_iris()
X = iris.data
y = iris.target

X = X.astype(int) # 정수형으로 만들어서  카테고리 타입으로 만듬

In [36]:
chi2_selector = SelectKBest(chi2, k = 2)
X_kbest = chi2_selector.fit_transform(X, y)